## 1. Data 불러오기

In [1]:
import pandas as pd

In [2]:
# 데이터 불러오기
df = pd.read_csv('../../data/english_data_including_content_and_area_of_thumbnailtext.csv', usecols = [6, 8, 9], encoding='UTF-8')
df.head(3)

,view,thumbnail_text_content,thumbnail_text_area
0,101580,Sweets How,0.11259
1,1147867,NaN,0.00000
2,429219,NaN,0.00000


In [3]:
data_cat = df.copy()           
#print(data_cat.info())           
data_cat['views'] = (data_cat['view'].floordiv(1000))
data_cat['content'] = data_cat['thumbnail_text_content'].fillna('  ')
data_cat['area'] = (data_cat['thumbnail_text_area']*10000).astype('int64')
data_cat.drop(['view', 'thumbnail_text_content', 'thumbnail_text_area'], axis='columns', inplace=True)
data_cat.head(4)

,views,content,area
0,101,Sweets How,1125
1,1147,,0
2,429,,0
3,607,I 6AM school morning routine,1195


## 2. views preprocess
before one-hot encoding

In [4]:
data_cat['views'].max()

314441

In [5]:
import math # 구간 나눠주기 -> multiclass classification

for index, view in enumerate(data_cat['views']):
  # if view >= 50000:  data_cat['views'][index] =  50000
  if view >=10000:  data_cat['views'][index] =  5
  # elif view >=5000:  data_cat['views'][index] =  5000
  elif view >=1000:  data_cat['views'][index] =  4
  # elif view >=500:  data_cat['views'][index] =  500
  elif view >=100:  data_cat['views'][index] = 3
  # elif view >=50: data_cat['views'][index] =  50
  elif view >=10: data_cat['views'][index] = 2
  # elif view >=5: data_cat['views'][index] = 5
  # elif view >5: data_cat['views'][index] = math.floor(data_cat['views'][index]/5) * 5
  elif view > 0: data_cat['views'][index] = 1

<ipython-input-5-2cc45fe1c4ad>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=100:  data_cat['views'][index] = 3
<ipython-input-5-2cc45fe1c4ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=1000:  data_cat['views'][index] =  4
<ipython-input-5-2cc45fe1c4ad>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif view >=10: data_cat['views'][index] = 2
<ipython-input-5-2cc45fe1c4ad>:14: SettingWithCopyWarning

In [6]:
from collections import Counter

label = Counter(data_cat['views']).keys()
label = sorted(label)
print(label)
print(Counter(data_cat['views']))

[0, 1, 2, 3, 4, 5]
Counter({4: 1504, 3: 1426, 2: 710, 0: 521, 1: 259, 5: 198})


## 3. content preprocess (tokenization)

In [12]:
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
data_cat['content_tokens']=None
data_cat[:3]

,views,content,area,content_tokens
0,3,Sweets How,1125,None
1,4,,0,None
2,3,,0,None


In [15]:
for index in range(len(df)):
    data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])
data_cat[:3]

<ipython-input-15-89f23f830b8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])


,views,content,area,content_tokens
0,3,Sweets How,1125,"[Sweets, How]"
1,4,,0,[]
2,3,,0,[]


In [16]:
#방법1
'''
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
data_cat['content_tokens1']=None
for index in range(len(df)):
    data_cat['content_tokens1'][index] = [lm.lemmatize(w, pos="v") for w in data_cat['content_tokens'][index]]
'''

'\nfrom nltk.stem import WordNetLemmatizer\nlm = WordNetLemmatizer()\ndata_cat[\'content_tokens1\']=None\nfor index in range(len(df)):\n    data_cat[\'content_tokens1\'][index] = [lm.lemmatize(w, pos="v") for w in data_cat[\'content_tokens\'][index]]\n'

In [17]:
#방법2
from nltk.stem import PorterStemmer
st1 = PorterStemmer()

#data_cat['content_tokens']=None

for index in range(len(df)):
    data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]

<ipython-input-17-a3a3f5d3aee5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]


In [18]:
#방법3
'''
from nltk.stem import LancasterStemmer
st2 = LancasterStemmer()

data_cat['content_tokens3']=None

for index in range(len(df)):
    data_cat['content_tokens3'][index] = [st2.stem(w) for w in data_cat['content_tokens'][index]]
'''

"\nfrom nltk.stem import LancasterStemmer\nst2 = LancasterStemmer()\n\ndata_cat['content_tokens3']=None\n\nfor index in range(len(df)):\n    data_cat['content_tokens3'][index] = [st2.stem(w) for w in data_cat['content_tokens'][index]]\n"

In [19]:
#방법 1,2,3 중 선택
'''
data_cat['content_tokens']=data_cat['content_tokens2']
data_cat.drop(['content', 'content_tokens1', 'content_tokens2', 'content_tokens3'], axis='columns', inplace=True)
data_cat[:3]
'''

"\ndata_cat['content_tokens']=data_cat['content_tokens2']\ndata_cat.drop(['content', 'content_tokens1', 'content_tokens2', 'content_tokens3'], axis='columns', inplace=True)\ndata_cat[:3]\n"

In [20]:
data_cat[:4]

,views,content,area,content_tokens
0,3,Sweets How,1125,"[sweet, how]"
1,4,,0,[]
2,3,,0,[]
3,3,I 6AM school morning routine,1195,"[i, 6am, school, morn, routin]"


## 4. make csv file

In [21]:
data_cat.to_csv("../../data/english_humbnailtext_preprocessed.csv", header=True, index=False)
data_cat

,views,content,area,content_tokens
0,3,Sweets How,1125,"[sweet, how]"
1,4,,0,[]
2,3,,0,[]
3,3,I 6AM school morning routine,1195,"[i, 6am, school, morn, routin]"
4,2,,0,[]
...,...,...,...,...
4613,2,,0,[]
4614,0,,0,[]
4615,3,PLATEK,1072,[platek]
4616,3,PIATER,1087,[piater]
